# пословный машинный перевод!

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import KeyedVectors
from sklearn.linear_model import LinearRegression

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 0.9999998807907104),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095538258552551),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.000000238418579),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810409307479858),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256801068782806),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632379829883575),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.22305874526500702),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796569228172302)]

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, y_test = load_word_pairs("ukr_rus.test.txt")

## Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [8]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [9]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.854128360748291),
 ('июнь', 0.8411200046539307),
 ('март', 0.8396992087364197),
 ('сентябрь', 0.8359866738319397),
 ('февраль', 0.8329296112060547),
 ('октябрь', 0.8311845064163208),
 ('ноябрь', 0.8278921246528625),
 ('июль', 0.8234526515007019),
 ('август', 0.8120498657226562),
 ('декабрь', 0.8039001226425171)]

In [10]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0

    for i, (_, ru) in enumerate(pairs):
        sim_dict = dict(ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn))
        if ru in sim_dict.keys():
            num_matches += 1
    precision_val = num_matches / len(pairs)
    
    return precision_val

In [11]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [12]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, y_test) == 1.0

In [13]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [14]:
assert precision_top1 >= 0.635
assert precision_top5 >= 0.813
print(precision_top1, precision_top5, sep='\n')

0.6356589147286822
0.813953488372093


## Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [17]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    X_t_Y = np.matmul(X_train.T, Y_train)
    U, _, Vt = np.linalg.svd(X_t_Y, full_matrices=False)
    W_new = np.matmul(U, Vt)
    
    return W_new

In [18]:
W  = learn_transform(X_train, y_train)

In [19]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)], topn=10)

[('апрель', 0.823790967464447),
 ('сентябрь', 0.8049710988998413),
 ('март', 0.802565336227417),
 ('июнь', 0.8021842837333679),
 ('октябрь', 0.8001735210418701),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.790810763835907),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686372399330139)]

In [20]:
precision_top1 = precision(uk_ru_test, np.matmul(X_test, W))
precision_top5 = precision(uk_ru_test, np.matmul(X_test, W), 5)

assert precision_top1 >= 0.653
assert precision_top5 >= 0.824
print(precision_top1, precision_top5, sep='\n')

0.6537467700258398
0.8242894056847545


___________________________________________

## Пишем переводчик

In [21]:
with open("fairy_tale.txt", "r", encoding='UTF-8') as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [22]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    char_dict = {'і': 'и', 'ї': 'е', 'є': 'e'} # test
    marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_'''
    traslated_sentence = ''
    for word in sentence.split():
        try:
            translate = ru_emb.most_similar([np.matmul(uk_emb[word], W)], topn=1)[0][0]
        except KeyError:
            for character in word:
                if character in char_dict.keys():
                    word = word.replace(character, char_dict[character])
            translate = word
            
        if word in list(marks): # удаление пробелов перед пунктуацией
            traslated_sentence = traslated_sentence.rstrip() 
        traslated_sentence = traslated_sentence + translate + ' '
        
    return traslated_sentence.rstrip()  

In [23]:
assert translate(".") == "."
assert translate("1, 3") == "1, 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [24]:
for sentence in uk_sentences[:15]:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка– сестричка и волк– панибрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка, че и пошла раз к одние бабы добувать огня; вошла во избу че и говорит:" хороший день тебе, бабушку!

src: дай мені огня " .
dst: дай мне огня".

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком, сладкий, че и согнула, чтобы он прохолов; а лисичка ой и пидгледала, че токмо что бабка качнулась во печь, чтобы достать огня, то лисичка сейчас ухватила пирожок че и деру со хаты, че, пробежать, весь мак со его виела, а туда мусора наложила.

src: пр